In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
import matplotlib.pyplot as plt

# Configuration
CONFIG = {
    'input_csv': '../../data/daily_stock_price/filtered_top25sp_stocks.csv',
    'models_folder': 'stock_models',
    'output_csv': 'evaluation_results.csv',
    'saliency_folder': 'lstm_saliency_outputs',
    'features': ['open', 'high', 'low', 'close', 'volume'],
    'target': 'close',
    'sequence_length': 60,
    'train_split': 0.8,
    'random_seed': 42,
    'lstm_epochs': 50,
    'batch_size': 32,
    'validation_split': 0.1
}

np.random.seed(CONFIG['random_seed'])
os.makedirs(CONFIG['models_folder'], exist_ok=True)
os.makedirs(CONFIG['saliency_folder'], exist_ok=True)
output_csv_path = os.path.join(CONFIG['models_folder'], CONFIG['output_csv'])

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length, CONFIG['features'].index(CONFIG['target'])])
    return np.array(X), np.array(y)

def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape, name="lstm_1"),
        Dropout(0.2),
        LSTM(50, return_sequences=False, name="lstm_2"),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model

def compute_saliency_and_hidden(stock):
    model_path = os.path.join(CONFIG['models_folder'], f"{stock}_lstm_model.h5")
    scaler_path = os.path.join(CONFIG['models_folder'], f"{stock}_scaler.pkl")
    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        return None, None

    model = load_model(model_path, custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
    scaler = joblib.load(scaler_path)
    df = pd.read_csv(CONFIG['input_csv'])
    stock_data = df[df['symbol'] == stock][CONFIG['features']].values
    scaled_data = scaler.transform(stock_data)
    X, y = create_sequences(scaled_data, CONFIG['sequence_length'])
    train_size = int(len(X) * CONFIG['train_split'])
    X_test = X[train_size:]

    X_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    X_tensor = tf.Variable(X_tensor)

    # Saliency
    with tf.GradientTape() as tape:
        tape.watch(X_tensor)
        preds = model(X_tensor)
    grads = tape.gradient(preds, X_tensor).numpy()
    saliency = np.abs(grads).mean(axis=(0, 1))

    # Save per-stock saliency CSV
    saliency_df = pd.DataFrame([saliency], columns=CONFIG['features'])
    saliency_df.to_csv(os.path.join(CONFIG['saliency_folder'], f"{stock}_saliency.csv"), index=False)

    # Hidden Layer Activations
    intermediate_layer_model = tf.keras.Model(inputs=model.input, outputs=[model.get_layer("lstm_1").output, model.get_layer("lstm_2").output])
    lstm_outputs = intermediate_layer_model.predict(X_tensor, verbose=0)
    lstm1_activations = np.mean(lstm_outputs[0], axis=(0, 1))  # avg over time and batch
    lstm2_activations = np.mean(lstm_outputs[1], axis=0)

    hidden_df = pd.DataFrame({"lstm1_neuron": lstm1_activations, "lstm2_neuron": lstm2_activations})
    hidden_df.to_csv(os.path.join(CONFIG['saliency_folder'], f"{stock}_hidden_activations.csv"), index=False)

    return saliency_df, hidden_df

def train_models():
    df = pd.read_csv(CONFIG['input_csv'])
    stocks = df['symbol'].unique()
    for stock in stocks:
        print(f"Training model for {stock}...")
        stock_data = df[df['symbol'] == stock][CONFIG['features']].values
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(stock_data)
        X, y = create_sequences(scaled_data, CONFIG['sequence_length'])
        train_size = int(len(X) * CONFIG['train_split'])
        X_train, y_train = X[:train_size], y[:train_size]
        model = build_lstm_model((CONFIG['sequence_length'], len(CONFIG['features'])))
        model.fit(X_train, y_train, epochs=CONFIG['lstm_epochs'], batch_size=CONFIG['batch_size'],
                  validation_split=CONFIG['validation_split'], verbose=1)
        model.save(os.path.join(CONFIG['models_folder'], f"{stock}_lstm_model.h5"))
        joblib.dump(scaler, os.path.join(CONFIG['models_folder'], f"{stock}_scaler.pkl"))


def evaluate_all_stocks():
    df = pd.read_csv(CONFIG['input_csv'])
    stocks = df['symbol'].unique()
    results = []
    saliency_all = []
    hidden_all = []

    for stock in stocks:
        print(f"Evaluating {stock}...")
        model_path = os.path.join(CONFIG['models_folder'], f"{stock}_lstm_model.h5")
        scaler_path = os.path.join(CONFIG['models_folder'], f"{stock}_scaler.pkl")
        if not os.path.exists(model_path) or not os.path.exists(scaler_path):
            continue

        try:
            model = load_model(model_path, custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
            scaler = joblib.load(scaler_path)
        except Exception as e:
            print(f"Error loading model/scaler for {stock}: {e}")
            continue

        stock_data = df[df['symbol'] == stock][CONFIG['features']].values
        scaled_data = scaler.transform(stock_data)
        X, y = create_sequences(scaled_data, CONFIG['sequence_length'])
        train_size = int(len(X) * CONFIG['train_split'])
        X_test, y_test = X[train_size:], y[train_size:]
        y_pred_scaled = model.predict(X_test, verbose=0)

        y_test_full = np.zeros((len(y_test), len(CONFIG['features'])))
        y_pred_full = np.zeros((len(y_pred_scaled), len(CONFIG['features'])))
        idx = CONFIG['features'].index(CONFIG['target'])
        y_test_full[:, idx] = y_test
        y_pred_full[:, idx] = y_pred_scaled.flatten()
        y_test_inv = scaler.inverse_transform(y_test_full)[:, idx]
        y_pred_inv = scaler.inverse_transform(y_pred_full)[:, idx]

        results.append({
            'Stock': stock,
            'MSE': mean_squared_error(y_test_inv, y_pred_inv),
            'RMSE': np.sqrt(mean_squared_error(y_test_inv, y_pred_inv)),
            'MAE': mean_absolute_error(y_test_inv, y_pred_inv),
            'R2': r2_score(y_test_inv, y_pred_inv)
        })

        saliency_df, hidden_df = compute_saliency_and_hidden(stock)
        if saliency_df is not None:
            saliency_df['Stock'] = stock
            hidden_df['Stock'] = stock
            saliency_all.append(saliency_df)
            hidden_all.append(hidden_df)

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv_path, index=False)
    display(results_df)

    # Save combined saliency and hidden activations
    if saliency_all:
        all_saliency = pd.concat(saliency_all, ignore_index=True)
        all_saliency.groupby('Stock').mean().mean().sort_values(ascending=False).to_frame(name='Mean Saliency').to_csv(
            os.path.join(CONFIG['saliency_folder'], 'saliency_summary.csv'))

    if hidden_all:
        all_hidden = pd.concat(hidden_all, ignore_index=True)
        all_hidden.to_csv(os.path.join(CONFIG['saliency_folder'], 'hidden_activation_summary.csv'), index=False)

    # Plot saliency
    mean_saliency = all_saliency.drop(columns='Stock').mean().sort_values(ascending=False)
    plt.figure(figsize=(8, 5))
    mean_saliency.plot(kind='barh')
    plt.title("LSTM Feature Importance via Saliency")
    plt.xlabel("Mean Saliency")
    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['saliency_folder'], 'saliency_barplot.png'))
    plt.show()

def main():
    train_models()
    evaluate_all_stocks()

main()


/Users/kwklun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Training model for ABBV...
Epoch 1/50


2025-05-02 03:02:38.747175: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-05-02 03:02:38.747200: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-05-02 03:02:38.747205: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2025-05-02 03:02:38.747220: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-02 03:02:38.747229: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/kwklun/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequ

68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.0118 - val_loss: 0.0011
Epoch 2/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 6.4303e-04 - val_loss: 7.7657e-04
Epoch 3/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 5.0659e-04 - val_loss: 0.0024
Epoch 4/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 4.1104e-04 - val_loss: 0.0022
Epoch 5/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 4.2714e-04 - val_loss: 4.7062e-04
Epoch 6/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 2.7772e-04 - val_loss: 0.0013
Epoch 7/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 2.9609e-04 - val_loss: 6.7654e-04
Epoch 8/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 2.6722e-04 - val_loss: 6.0556e-04
Epoch 9/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 2.5419e-04 - val_loss: 3.2188e-04
Epoch 10/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 2.5545e-04 - val_loss: 9.1216e-04
Epoch 11/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 2.8994e-04 - val_loss: 3.1519e-04
Epoch 

Training model for GOOGL...
Epoch 1/50


...

Training model for WMT...
Epoch 1/50


/Users/kwklun/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


297/297 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - loss: 1.2022e-04 - val_loss: 2.5066e-05
Epoch 2/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 2.4802e-05 - val_loss: 1.0554e-05
Epoch 3/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 2.5273e-05 - val_loss: 7.7075e-05
Epoch 4/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 2.2436e-05 - val_loss: 9.3179e-06
Epoch 5/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 2.6079e-05 - val_loss: 2.0086e-05
Epoch 6/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 2.6523e-05 - val_loss: 1.7256e-05
Epoch 7/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 2.2914e-05 - val_loss: 9.4580e-06
Epoch 8/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 2.1444e-05 - val_loss: 2.1896e-05
Epoch 9/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 2.2779e-05 - val_loss: 7.7678e-06
Epoch 10/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 2.4096e-05 - val_loss: 1.2465e-05
Epoch 11/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss

Evaluating ABBV...


AttributeError: The layer sequential has never been called and thus has no defined input.

In [16]:
import pandas as pd
import numpy as np
import os
import joblib
from tensorflow.keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from contextlib import redirect_stdout
import io

def compute_saliency_and_hidden(stock):
    model_path = os.path.join(CONFIG['models_folder'], f"{stock}_lstm_model.h5")
    scaler_path = os.path.join(CONFIG['models_folder'], f"{stock}_scaler.pkl")
    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        print(f"Model or scaler not found for {stock}")
        return None, None

    # Load model and scaler
    model = load_model(model_path, custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
    model.compile(optimizer='adam', loss='mse')  # Recompile to avoid warning
    scaler = joblib.load(scaler_path)

    # Prepare input data
    df = pd.read_csv(CONFIG['input_csv'])
    stock_data = df[df['symbol'] == stock][CONFIG['features']].values
    scaled_data = scaler.transform(stock_data)
    X, y = create_sequences(scaled_data, CONFIG['sequence_length'])
    train_size = int(len(X) * CONFIG['train_split'])
    X_test = X[train_size:]

    # Prepare input tensor
    X_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    X_tensor = tf.Variable(X_tensor)

    # Initialize model: Build and evaluate
    try:
        # Explicitly build the model with input shape
        input_shape = (None, CONFIG['sequence_length'], len(CONFIG['features']))
        model.build(input_shape)
        print(f"Built model for {stock} with input shape: {input_shape}")
        print(f"Model input shape after build: {model.input_shape}")

        # Evaluate model to initialize metrics and ensure input is defined
        model.predict(X_tensor[:1], verbose=0)  # Single batch prediction
        print(f"Model evaluated for {stock}, input shape: {model.input_shape}")
    except Exception as e:
        print(f"Failed to initialize model for {stock}: {e}")
        return None, None

    # Compute saliency
    with tf.GradientTape() as tape:
        tape.watch(X_tensor)
        preds = model(X_tensor)
    grads = tape.gradient(preds, X_tensor).numpy()
    saliency = np.abs(grads).mean(axis=(0, 1))
    saliency_df = pd.DataFrame([saliency], columns=CONFIG['features'])
    saliency_df['Stock'] = stock  # Add stock identifier

    # Visualize saliency for this stock
    plt.figure(figsize=(10, 6))
    saliency_df.drop(columns='Stock').iloc[0].sort_values(ascending=False).plot(kind='barh')
    plt.title(f"Saliency for {stock}")
    plt.xlabel("Saliency Value")
    plt.ylabel("Features")
    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['saliency_folder'], f"{stock}_saliency_plot.png"))
    plt.close()

    # Extract hidden layer activations
    try:
        # Debug: Print model summary to verify layer names
        summary_io = io.StringIO()
        with redirect_stdout(summary_io):
            model.summary()
        print(f"Model summary for {stock}:\n{summary_io.getvalue()}")

        # Try accessing lstm_1 and lstm_2
        intermediate_layer_model = tf.keras.Model(
            inputs=model.input,
            outputs=[
                model.get_layer("lstm_1").output,
                model.get_layer("lstm_2").output
            ]
        )
        lstm_outputs = intermediate_layer_model.predict(X_tensor, verbose=0)
        lstm1_activations = np.mean(lstm_outputs[0], axis=(0, 1))  # avg over time and batch
        lstm2_activations = np.mean(lstm_outputs[1], axis=0)

        hidden_df = pd.DataFrame({
            "lstm1_neuron": lstm1_activations,
            "lstm2_neuron": lstm2_activations
        })
        hidden_df['Stock'] = stock  # Add stock identifier
        hidden_df.to_csv(os.path.join(CONFIG['saliency_folder'], f"{stock}_hidden_activations.csv"), index=False)
    except Exception as e:
        print(f"Failed to extract hidden activations for {stock}: {e}")
        # Print model summary again in case of error
        print(f"Model architecture on error for {stock}:\n{summary_io.getvalue()}")
        # Fallback: Dynamic LSTM layer selection
        try:
            lstm_layers = [layer for layer in model.layers if isinstance(layer, tf.keras.layers.LSTM)]
            if len(lstm_layers) < 2:
                print(f"Not enough LSTM layers in model for {stock}. Found: {[layer.name for layer in lstm_layers]}")
                return saliency_df, None
            intermediate_layer_model = tf.keras.Model(
                inputs=model.input,
                outputs=[lstm_layers[0].output, lstm_layers[1].output]
            )
            lstm_outputs = intermediate_layer_model.predict(X_tensor, verbose=0)
            lstm1_activations = np.mean(lstm_outputs[0], axis=(0, 1))
            lstm2_activations = np.mean(lstm_outputs[1], axis=0)

            hidden_df = pd.DataFrame({
                "lstm1_neuron": lstm1_activations,
                "lstm2_neuron": lstm2_activations
            })
            hidden_df['Stock'] = stock
            hidden_df.to_csv(os.path.join(CONFIG['saliency_folder'], f"{stock}_hidden_activations.csv"), index=False)
        except Exception as e2:
            print(f"Dynamic LSTM layer selection failed for {stock}: {e2}")
            return saliency_df, None

    return saliency_df, hidden_df

def evaluate_only():
    df = pd.read_csv(CONFIG['input_csv'])
    stocks = df['symbol'].unique()
    results = []
    saliency_all = []
    hidden_all = []

    for stock in stocks:
        print(f"Evaluating {stock}...")
        model_path = os.path.join(CONFIG['models_folder'], f"{stock}_lstm_model.h5")
        scaler_path = os.path.join(CONFIG['models_folder'], f"{stock}_scaler.pkl")
        if not os.path.exists(model_path) or not os.path.exists(scaler_path):
            print(f"Skipping {stock}: Model or scaler not found")
            continue

        try:
            model = load_model(model_path, custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
            model.compile(optimizer='adam', loss='mse')  # Recompile to avoid warning
            scaler = joblib.load(scaler_path)
        except Exception as e:
            print(f"Error loading model/scaler for {stock}: {e}")
            continue

        stock_data = df[df['symbol'] == stock][CONFIG['features']].values
        scaled_data = scaler.transform(stock_data)
        X, y = create_sequences(scaled_data, CONFIG['sequence_length'])
        train_size = int(len(X) * CONFIG['train_split'])
        X_test, y_test = X[train_size:], y[train_size:]
        y_pred_scaled = model.predict(X_test, verbose=0)

        y_test_full = np.zeros((len(y_test), len(CONFIG['features'])))
        y_pred_full = np.zeros((len(y_pred_scaled), len(CONFIG['features'])))
        idx = CONFIG['features'].index(CONFIG['target'])
        y_test_full[:, idx] = y_test
        y_pred_full[:, idx] = y_pred_scaled.flatten()
        y_test_inv = scaler.inverse_transform(y_test_full)[:, idx]
        y_pred_inv = scaler.inverse_transform(y_pred_full)[:, idx]

        results.append({
            'Stock': stock,
            'MSE': mean_squared_error(y_test_inv, y_pred_inv),
            'RMSE': np.sqrt(mean_squared_error(y_test_inv, y_pred_inv)),
            'MAE': mean_absolute_error(y_test_inv, y_pred_inv),
            'R2': r2_score(y_test_inv, y_pred_inv)
        })

        saliency_df, hidden_df = compute_saliency_and_hidden(stock)
        if saliency_df is not None:
            saliency_all.append(saliency_df)
        if hidden_df is not None:
            hidden_all.append(hidden_df)

    # Save evaluation results
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv_path, index=False)
    print("Evaluation Results:")
    print(results_df)

    # Process and visualize combined saliency
    if saliency_all:
        all_saliency = pd.concat(saliency_all, ignore_index=True)
        all_saliency.to_csv(os.path.join(CONFIG['saliency_folder'], 'all_saliency.csv'), index=False)

        # Compute mean saliency across all stocks
        mean_saliency = all_saliency.drop(columns='Stock').mean().sort_values(ascending=False)
        mean_saliency_df = mean_saliency.to_frame(name='Mean Saliency')
        mean_saliency_df.to_csv(os.path.join(CONFIG['saliency_folder'], 'model_saliency_average.csv'))

        # Visualize combined saliency
        plt.figure(figsize=(10, 6))
        mean_saliency.plot(kind='barh')
        plt.title("Mean Saliency Across All Stocks")
        plt.xlabel("Mean Saliency")
        plt.ylabel("Features")
        plt.tight_layout()
        plt.savefig(os.path.join(CONFIG['saliency_folder'], 'combined_saliency_plot.png'))
        plt.close()

    # Save combined hidden activations
    if hidden_all:
        all_hidden = pd.concat(hidden_all, ignore_index=True)
        all_hidden.to_csv(os.path.join(CONFIG['saliency_folder'], 'hidden_activation_summary.csv'), index=False)

    return results_df, all_saliency if saliency_all else None, all_hidden if hidden_all else None

# Call to run evaluation
if __name__ == "__main__":
    results_df, all_saliency, all_hidden = evaluate_only()

Evaluating ABBV...


Built model for ABBV with input shape: (None, 60, 5)
Model input shape after build: (None, 60, 5)
Model evaluated for ABBV, input shape: (None, 60, 5)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 60, 50)         │        11,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,701 (127.74 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 60, 50)         │        11,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_48 (Dropout)            │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_49 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,701 (127.74 KB)

 Trainable params: 32,701 (127.74 KB)

 Non-trainable params: 0 (0.00 B)

Model summary for WMT:

Failed to extract hidden activations for WMT: The layer sequential_24 has never been called and thus has no defined input.
Model architecture on error for WMT:

Dynamic LSTM layer selection failed for WMT: The layer sequential_24 has never been called and thus has no defined input.
Evaluation Results:
    Stock           MSE        RMSE        MAE        R2
0    ABBV     14.969746    3.869076   2.834885  0.967233
1   GOOGL     24.130472    4.912278   3.678896  0.967547
2    GOOG    265.232886   16.285972  14.652526  0.650809
3    AMZN    633.073856   25.160959  22.100684  0.523061
4    AAPL    716.394607   26.765549  17.178600  0.837244
5     BAC      0.948593    0.973957   0.796138  0.989163
6   BRK-B    171.352905   13.090184   9.359313  0.975302
7    AVGO    118.186201   10.871348   6.293096  0.960161
8    COST   4922.394531   70.159779  43.457655  0.903523
9     XOM     95.748739    9.785128   5.155087  0.801611
10     HD   1701.470078   41.248880  32.403153